# Train a baseline model for Subchallenge 1

We'll train an elastic net on RNASeq data to predict "General Response across Drugs" (GRD). Specifically:

- Select the 1000 highest variance genes (note that [1] used gene sets)
- Average AUCs per-patient to get GRD
- Train with 10-fold cross-validation
- alphas & l1_ratios borrowed from [1]

[1] Barretina, Jordi, Giordano Caponigro, Nicolas Stransky, Kavitha Venkatesan, Adam A. Margolin, Sungjoon Kim, Christopher J. Wilson, et al. “The Cancer Cell Line Encyclopedia Enables Predictive Modelling of Anticancer Drug Sensitivity.” Nature 483, no. 7391 (March 2012): 603–7. https://doi.org/10.1038/nature11003.


### First, download training data.

This only needs to be run once, to populate the `training/` directory.

In [ ]:
%pip install synapseclient

import getpass
import pandas
import synapseclient
import synapseutils

# TODO: Change the synapse directory once the project launches.

syn = synapseclient.Synapse()
syn.login(input(prompt="Enter Synapse Username"), getpass.getpass("Enter Synapse Password"))
downloaded_files = synapseutils.syncFromSynapse(syn, 'syn21212904', path='training') 

### Now, load the data, and train a model!

In [ ]:
import pandas
rnaseq = pandas.read_csv('training/rnaseq.csv')
clinical = pandas.read_csv('training/clinical.csv')
aucs = pandas.read_csv('training/aucs.csv')

In [ ]:
from util import TransposeRnaSeqTable

specimens = TransposeRnaSeqTable(rnaseq)
selected_genes = specimens.var().nlargest(1000).index.tolist()

In [ ]:
print("%.1f%% of (inhibitor, specimen) pairs have AUCs." % (
    100 * aucs.shape[0] / float(len(aucs.inhibitor.unique()) * len(aucs.lab_id.unique()))))

In [ ]:
grds = aucs.groupby('lab_id').auc.mean()

In [ ]:
from matplotlib import pyplot
import numpy
import seaborn
from sklearn.linear_model import RidgeCV

# Normalize each specimen.
X = specimens
X = X.div(numpy.linalg.norm(X, axis=1), axis=0)
X = X[selected_genes]

# Compute z-score.
gene_mean = X.mean(axis=0)
gene_std = X.std(axis=0)
X = (X - gene_mean) / gene_std

# Do the fit.
alphas = numpy.logspace(-1, 4, num=30)
regr = RidgeCV(alphas=alphas, store_cv_values=True)
regr = regr.fit(X, grds[X.index])

# Plot.
errors = numpy.sqrt(regr.cv_values_.mean(axis=0))
seaborn.scatterplot(x=alphas, y=errors, label='ridge training error')
pyplot.axhline(y=grds.std(), label='grds variance\n(zero skill prediction)')

# Annotate.
pyplot.xlabel('alpha')
pyplot.ylabel('Cross-Validation RMSE [GRD]')
pyplot.title('Ridge Regression training error\nOptimal alpha: %0.1f\nMin error: %.3f' % (regr.alpha_, min(errors)))
pyplot.legend()

### Try pickling it.

In [ ]:
pkl_1_out = pandas.DataFrame({
    'gene': selected_genes,
    'gene_mean': gene_mean,
    'gene_std': gene_std,
    'fit': regr.coef_,
})
pkl_2_out = pandas.DataFrame({
    'parameter': ['intercept'],
    'value': [regr.intercept_],
})
pkl_1_out.to_csv('model/pkl_1.csv', index=False)
pkl_2_out.to_csv('model/pkl_2.csv', index=False)

### Look at predictions vs goldstandard for training data

Assumes predictions are in `output/aucs.csv`.

In [ ]:
indices = ['lab_id', 'inhibitor']
groundtruth = pandas.read_csv('training/aucs.csv').set_index(indices)
predictions = pandas.read_csv('output/aucs.csv').set_index(indices)
predictions_and_groundtruth = groundtruth.join(
    predictions, lsuffix='_groundtruth', rsuffix='_prediction')

In [ ]:
seaborn.scatterplot(
    x='auc_groundtruth',
    y='auc_prediction',
    data=predictions_and_groundtruth,
    alpha=0.05)
pyplot.title('SC1 baseline predictor')